In [8]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
from SimLib import config_sim as conf
from SimLib import sipm_mapping as DAQ
import sys
#sys.path.append("/home/viherbos/GITHUB/PETALO_analysis")
import fit_library
import scipy.signal as sc
import itertools as it
import multiprocessing as mp
import time
from antea.reco.reco_functions import find_first_interactions_in_active as antea_ffia
#import TOF_library as T_lib
%matplotlib nbagg
%reload_ext autoreload
%autoreload 2   


class MOVIE(object):
    """ Create a movie-like register of the gamma1+gamma2 TRUE events
    """
    def __init__(self, path, filename, SiPM):
        """ path and filename of .h5 data file
            SiPM = [first_sipm, n_sipms]
        """
        file = path + filename
        particles = pd.read_hdf(file,key='MC/particles')
        hits      = pd.read_hdf(file,key='MC/hits')
        self.keys_particles = particles.columns
        self.keys_hits      = hits.columns
        self.particles      = particles
        self.hits           = hits
        self.tof_wave       = np.array(pd.read_hdf(file,key='MC/tof_waveforms'))
        self.n_sipms        = SiPM['n_sipms']
        self.first_sipm     = SiPM['first_sipm']    
        
        json_path         = "/volumedisk0/home/viherbos/DAQ_data/"
        json_filename     = "CUBE"
        SIM_CONT          = conf.SIM_DATA(filename=json_path+json_filename+".json",read=True)
        data              = SIM_CONT.data
        L1_Slice, Matrix_I, Matrix_O, topo = DAQ.SiPM_Mapping(data,data['L1']['map_style'])
        self.Matrix_O     = Matrix_O
        
    def true_info(self,event):
        """ Gets data from given event
        """
        particles_i = self.particles[self.particles.event_id==event]
        hits_i      = self.hits[self.hits.event_id==event]
        return antea_ffia(particles_i, hits_i)
    
    def process_event(self,event, acc_time):
        """ acc_time: Length of Sliding Window Filter (in time_bins)
        """
        event_select    = np.argwhere(self.tof_wave[:,0]==event)
        event_tof       = self.tof_wave[event_select[:,0],1:]
        event_tof[:,0]  = event_tof[:,0]*-1-self.first_sipm
        # SiPM  |  Timebin  |  Charge

        time_length = np.max(event_tof[:,1])
        pe_table = np.zeros((time_length+1,self.n_sipms))
        
        for i in range(event_tof.shape[0]):
            pe_table[event_tof[i,1],event_tof[i,0]] = event_tof[i,2]
        
        T_photon_event = np.sum(pe_table,axis=1)
        # Sliding Window over Total Photons(time) for this event (computes slope of photon signal)
        
        slope          = np.convolve(T_photon_event,np.ones(acc_time))
        max_slope_time = np.argmax(slope) 
        # time of max arrival of photons/acc_time
                
        photon_acc  = np.cumsum(pe_table,axis=0)
        
        # We get the accumulated photons in to moments, highest gradient of photons and final photo
        #true_data = np.hstack([event,np.array(self.true_info(event))])
        #info1 = np.hstack([true_data, photon_acc[max_slope_time,:]])
        #info2 = np.hstack([true_data, photon_acc[-1,:]])
        
        event_info = np.vstack([photon_acc[max_slope_time,:],photon_acc[-1,:]])
        
        return event_info   
        

i = 0
name = "petit_ring_tof_all_tables"
path = "/volumedisk0/home/paolafer/vicente/"
file_name= name + "."+str(i).zfill(3)+".pet.h5"
i=0
A = MOVIE(path,file_name,{'first_sipm':1000,'n_sipms':3500})